## Задание 5 - 7

Загружаем необходимые библиотеки (нам понадобятся OS, psycopg2, numpy и pandas)

In [1]:
import datetime
import logging
import os
import psycopg2
from subprocess import call

import numpy as np
import pandas as pd


Настраиваем подключение к БД

In [2]:
params = {
    "host": '0.0.0.0',
    "port": '5433',
    "user": 'postgres'
}
conn = psycopg2.connect(**params)

Настраиваем cursor с помощью cursor мы будем раюотаь с SQL

In [3]:
psycopg2.extensions.register_type(
    psycopg2.extensions.UNICODE,
    conn
)
conn.set_isolation_level(
    psycopg2.extensions.ISOLATION_LEVEL_AUTOCOMMIT
)
cursor = conn.cursor()

## Запрос 1

Вывести количество жанров книг

эта инфармация поможет нам ариентироваться в данных которые мы будем получать в дальнейших запросах

In [4]:
zero_query = (
        """
            SELECT * FROM library;
        """ 
)

In [5]:
cursor.execute(zero_query)
q_data0 = [a for a in cursor.fetchall()]
conn.commit()

In [6]:
q_data0 

[(1, 'science fiction'),
 (2, 'non fiction'),
 (3, 'historical novel'),
 (4, 'adventures'),
 (5, 'romantic'),
 (6, 'horror'),
 (7, 'drama '),
 (8, 'documental '),
 (9, 'detective')]

In [7]:
df_0 = pd.DataFrame(q_data0, columns =['id', 'genre'])

In [8]:
df_0

,id,genre
0,1,science fiction
1,2,non fiction
2,3,historical novel
3,4,adventures
4,5,romantic
5,6,horror
6,7,drama
7,8,documental
8,9,detective


In [9]:
df_0.dtypes

id        int64
genre    object
dtype: object

## Запрос 2

Выполняем первый аналитический запрос:
    
Вывести три самых популярных жанра литературы (genre) 
среди выборки из 251 читателя (разного возраста, пола, образование, сферы деятельности) 
данная выборка реперезентативна она, в ней представлен широкий круг респондентов
    
Полученные данные помогут понять какие жанры литературы сейчас самые читаемые, 
данная нформация в дальнейшем поможет фрагментировать рынок при построениии бизнесс моделей издательств 

In [10]:
first_query = (
        """
            WITH tmp_tab as 
                (SELECT * FROM library 
                    JOIN library_card ON library.book_id=library_card.book_genre_id)
                SELECT genre, COUNT(book_genre_id) as top_genre 
                    FROM tmp_tab GROUP BY genre ORDER BY top_genre DESC LIMIT 3;
        """ 
)

Выполняем запрос на стороне Postgres и результат выгружаем в Python

In [11]:
cursor.execute(first_query)
q_data1 = [a for a in cursor.fetchall()]
conn.commit()

проверяем что получилось (результат список list) 

In [12]:
q_data1

[('romantic', 39), ('adventures', 36), ('non fiction', 31)]

In [13]:
type(q_data1)

list

Создаем data frame для дальнейшей работы с pandas 

In [14]:
df_1 = pd.DataFrame(q_data1, columns =['genre', 'top_genre'])

In [15]:
df_1

,genre,top_genre
0,romantic,39
1,adventures,36
2,non fiction,31


## Запрос 3

Выполняем второй аналитический запрос:
Вывести топ-3 жанра которые читают женщины старше 50 лет

Полученные данные помогут понять какие жанры литературы популярны у женьщин старшего возраста 

In [16]:
second_query = (
        """
            WITH tmp_tab as 
            (SELECT * FROM library JOIN books_after_50 ON library.book_id=books_after_50.book_genre_id) 
            SELECT DISTINCT genre FROM tmp_tab WHERE age>50 and gender LIKE '%Female' LIMIT 3;
        """ 
)

In [17]:
cursor.execute(second_query)
q_data2 = [a for a in cursor.fetchall()]
conn.commit()

In [18]:
q_data2

[('adventures',), ('detective',), ('documental ',)]

In [19]:
df_2 = pd.DataFrame(q_data2, columns =['genre'])

In [20]:
df_2

,genre
0,adventures
1,detective
2,documental


## Запрос 4

Определяем минимальный возраст респондента (среди респондентов нашего опроса из запроса 1)

In [21]:
therd_query = (
        """
           SELECT Min(age) as min_age FROM employee;
        """ 
)

In [22]:
cursor.execute(therd_query)
q_data3 = [a for a in cursor.fetchall()]
conn.commit()

In [23]:
df_3 = pd.DataFrame(q_data3, columns =['min_age'])

In [24]:
df_3.head()

,min_age
0,17


## Запрос 5

Выполняем третий аналитический запрос: Вывести средний значения продолжительности чтения книг в зависимости от жанра

Эти данные помогут издателям понять сколько времени читатели разных жанров тратят времени на чтение книг, данную 
информацию можно использовать при принятии решения какое количиство страниц оптимально для книг представленных в каждом жанре

In [38]:
forth_query = (
        """
            WITH tmp_tab as 
            (SELECT * FROM library JOIN library_card ON library.book_id=library_card.book_genre_id) 
            SELECT genre, AVG(reading_time_days) as read_term FROM tmp_tab GROUP BY genre ORDER BY read_term DESC;
        """ 
)

In [39]:
cursor.execute(forth_query)
q_data4 = [a for a in cursor.fetchall()]
conn.commit()

In [40]:
df_4 = pd.DataFrame(q_data4, columns =['genre', 'avg_read_term'])

In [42]:
df_4

,genre,avg_read_term
0,historical novel,28.6800000000000000
1,drama,28.5000000000000000
2,adventures,27.1666666666666667
3,romantic,25.7692307692307692
4,horror,22.5200000000000000
5,science fiction,21.5217391304347826
6,detective,21.1153846153846154
7,non fiction,17.8387096774193548
8,documental,16.5000000000000000


In [55]:
j_query = (
        """
            SELECT * FROM library JOIN library_card ON library.book_id=library_card.book_genre_id;
        """ 
)

In [58]:
cursor.execute(j_query)
j_data4 = [a for a in cursor.fetchall()]
conn.commit()

In [60]:
df_j = pd.DataFrame(j_data4, columns = ['book_id', 'genre', 'nameid', 'book_genre_id', 'reading_time_days'])

In [62]:
df_j.head(12)

,book_id,genre,nameid,book_genre_id,reading_time_days
0,1,science fiction,1001,1,34
1,2,non fiction,1002,2,21
2,1,science fiction,1003,1,2
3,8,documental,1004,8,3
4,7,drama,1005,7,4
5,9,detective,1006,9,5
6,5,romantic,1007,5,6
7,3,historical novel,1008,3,7
8,2,non fiction,1009,2,10
9,1,science fiction,1010,1,12
